In [35]:
# !pip install -r ../exp_requirements.txt

In [36]:
# !pip uninstall virny -y

In [37]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


# Folktables GA Dataset With Random Nulls

## Import dependencies

In [4]:
import os
import copy

from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from virny.utils.custom_initializers import create_config_obj, create_models_metrics_dct_from_database_df
from virny.datasets.data_loaders import ACSIncomeDataset
from source.user_interfaces.experiment_interface import run_exp_iter_with_models_stress_testing
from source.error_injectors.random_nulls_injector_v2 import RandomNullsInjectorV2
from source.utils.custom_initializers import create_experiment_data_loader
from source.utils.db_functions import read_model_metric_dfs_from_db
from source.preprocessing.basic_preprocessing import get_null_imputer_preprocessor

from configs.constants import NUM_METRICS_COMPUTATION_RUNS, EXPERIMENT_SEEDS, TEST_SET_FRACTION
from configs.models_params_for_tuning import get_folktables_income_models_params_for_tuning

## Initialize input variables for the experiment

In [5]:
# ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'stress_testing_nulls'
DB_COLLECTION_NAME = f'exp_{EXPERIMENT_NAME}'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
COLUMNS_TO_TRANSFORM = ['SCHL', 'COW', 'MAR', 'OCCP', 'POBP', 'RELP', 'WKHP']
INJECTOR_CONFIG_LST = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
MAX_NUM_COLUMNS_TO_EFFECT = 3

In [6]:
data_loader = ACSIncomeDataset(state=['GA'], year=2018, with_nulls=False,
                               subsample_size=20_000, subsample_seed=42)
data_loader.X_data.head()

,SCHL,COW,MAR,OCCP,POBP,RELP,SEX,RAC1P,AGEP,WKHP
0,23,7,3,230,36,0,1,1,55,55.0
1,16,1,5,4110,13,2,2,1,20,35.0
2,16,4,3,4130,51,0,2,1,59,30.0
3,18,4,1,4020,13,0,1,2,43,40.0
4,14,1,1,8300,20,1,2,2,33,20.0


In [7]:
data_loader.full_df.shape

(20000, 10)

In [8]:
data_loader.full_df.isna().sum()

SCHL     0
COW      0
MAR      0
OCCP     0
POBP     0
RELP     0
SEX      0
RAC1P    0
AGEP     0
WKHP     0
dtype: int64

### Define a db writer and custom fields to insert into your database

In [9]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [10]:
custom_table_fields_dct = {
    'error_type': 'Random Nulls',
    'error_percentages': '0%,5%,10%,20%,30%,40%,50%',
    'preprocessing_technique': 'cat: mode, num: median',
}

In [11]:
import uuid

custom_table_fields_dct['session_uuid'] = '340a6ce1-3261-4b64-a083-d74dfaad97cf'
# custom_table_fields_dct['session_uuid'] = str(uuid.uuid4())
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  340a6ce1-3261-4b64-a083-d74dfaad97cf


### Create a metrics computation config object

In [12]:
config_yaml_path = 'experiment_config.yaml'
age_range = [i for i in range(30, 46)]
config_yaml_content = \
f"""
dataset_name: Folktables_Income_GA_2018
bootstrap_fraction: 0.8
n_estimators: 200
num_runs: 1
runs_seed_lst: [100]
sensitive_attributes_dct: {{'SEX': '1', 'RAC1P': '1', 'AGEP': {age_range}, 'SEX & RAC1P': None, 'RAC1P & AGEP': None}}
"""

with open(config_yaml_path, 'w', encoding='utf-8') as f:
    f.write(config_yaml_content)

In [13]:
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

### Define tuning parameter for models

In [14]:
# def get_models_params_for_tuning(models_tuning_seed):
#     return {
#         'DecisionTreeClassifier': {
#             'model': DecisionTreeClassifier(random_state=models_tuning_seed),
#             'params': {
#                 "max_depth": [20, 30],
#                 "min_samples_split" : [0.1],
#                 "max_features": ['sqrt'],
#                 "criterion": ["gini", "entropy"]
#             }
#         },
#         'LogisticRegression': {
#             'model': LogisticRegression(random_state=models_tuning_seed),
#             'params': {
#                 'penalty': ['l2'],
#                 'C' : [0.0001, 0.1, 1, 100],
#                 'solver': ['newton-cg', 'lbfgs'],
#                 'max_iter': [250],
#             }
#         },
#         # 'RandomForestClassifier': {
#         #     'model': RandomForestClassifier(random_state=models_tuning_seed),
#         #     'params': {
#         #         "max_depth": [6, 10],
#         #         "min_samples_leaf": [1],
#         #         "n_estimators": [50, 100],
#         #         "max_features": [0.6]
#         #     }
#         # },
#         # 'XGBClassifier': {
#         #     'model': XGBClassifier(random_state=models_tuning_seed, verbosity=0),
#         #     'params': {
#         #         'learning_rate': [0.1],
#         #         'n_estimators': [200],
#         #         'max_depth': [5, 7],
#         #         'lambda':  [10, 100]
#         #     }
#         # },
#         # 'KNeighborsClassifier': {
#         #     'model': KNeighborsClassifier(),
#         #     'params': {
#         #         'n_neighbors' : [5, 7, 9, 11, 13, 15, 25],
#         #         'weights' : ['uniform', 'distance'],
#         #         'metric' : ['minkowski', 'euclidean', 'manhattan']
#         #     }
#         # },
#         # 'MLPClassifier': {
#         #     'model': MLPClassifier(random_state=models_tuning_seed),
#         #     'params': {
#         #         'hidden_layer_sizes':[(100,), (100,100,), (100,50,100,)],
#         #         'activation': ['logistic', 'tanh', 'relu'],
#         #         'solver': ['lbfgs', 'sgd', 'adam'],
#         #         'learning_rate': ['constant', 'invscaling', 'adaptive']
#         #     }
#         # }
#     }

## Run experiments

### Experiment iteration 1

In [22]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filename = 'tuning_results_COMPAS_exp_iter_1_20230425__164903.csv'
tuned_params_df_path = os.path.join(SAVE_RESULTS_DIR_PATH, 'models_tuning', tuned_params_filename)
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

error_injector = RandomNullsInjectorV2(experiment_seed, columns_to_transform=COLUMNS_TO_TRANSFORM, row_idx_nulls_percentage=0.0,
                                       max_num_columns_to_effect=MAX_NUM_COLUMNS_TO_EFFECT)
models_params_for_tuning = get_folktables_income_models_params_for_tuning(experiment_seed)
# Create a transformed data loader
exp_iter_data_loader = create_experiment_data_loader(data_loader)
exp_iter_data_loader.columns_with_nulls = COLUMNS_TO_TRANSFORM
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_models_stress_testing(data_loader=exp_iter_data_loader,
                                        experiment_seed=experiment_seed,
                                        test_set_fraction=TEST_SET_FRACTION,
                                        db_writer_func=db_writer_func,
                                        error_injector=error_injector,
                                        injector_config_lst=INJECTOR_CONFIG_LST,
                                        preprocessor=preprocessor,
                                        models_params_for_tuning=models_params_for_tuning,
                                        metrics_computation_config=metrics_computation_config,
                                        custom_table_fields_dct=custom_table_fields_dct,
                                        with_tuning=True,
                                        tuned_params_df_path=None,
#                                         with_tuning=False,
#                                         tuned_params_df_path=tuned_params_df_path,
                                        save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                        verbose=True)

2023-04-25 19:50:37 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'error_percentages': '0%,5%,10%,20%,30%,40%,50%',
 'error_type': 'Random Nulls',
 'experiment_iteration': 'Exp_iter_1',
 'injector_config_lst': '[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]',
 'model_init_seed': 100,
 'preprocessing_technique': 'cat: mode, num: median',
 'session_uuid': '340a6ce1-3261-4b64-a083-d74dfaad97cf'}




2023-04-25 19:50:38 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 2917,  2234, 14396,  1781, 11102,   732,  7692, 10589, 16098,
             4920,  6601,  7611,  3825, 18755,  6862,  3847,  7256, 13711,
            12389,  9772],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 2917,  2234, 14396,  1781, 11102,   732,  7692, 10589, 16098,
             4920,  6601,  7611,  3825, 18755,  6862,  3847,  7256, 13711,
            12389,  9772],
           dtype='int64')
error_injector.seed --  102
transformed_X_test:
 SCHL     35
COW      21
MAR      23
OCCP     33
POBP     28
RELP     27
SEX       0
RAC1P     0
AGEP      0
WKHP     26
dtype: int64
error_injector.seed --  103
transformed_X_test:
 SCHL     69
COW      50
MAR      44
OCCP     61
POBP     54
RELP     56
SEX       0
RAC1P     0
AGEP      0
WKHP     58
dtype: int64
error_injector.seed --  104
transformed_X_test:
 SCHL     124
COW       97
MAR      111
OCCP     116
POBP     108
RELP     112
SEX

2023-04-25 19:59:53 experiment_interface.py INFO    : Models are tuned and saved to a file


2023/04/25, 19:59:53: Tuning for KNeighborsClassifier is finished [F1 score = 0.76822240575623, Accuracy = 0.7898333333333333]



Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/5 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 2

In [131]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filename = 'tuning_results_COMPAS_exp_iter_1_20230425__164903.csv'
tuned_params_df_path = os.path.join(SAVE_RESULTS_DIR_PATH, 'models_tuning', tuned_params_filename)
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

error_injector = RandomNullsInjectorV2(experiment_seed, columns_to_transform=COLUMNS_TO_TRANSFORM, row_idx_nulls_percentage=0.0,
                                       max_num_columns_to_effect=MAX_NUM_COLUMNS_TO_EFFECT)
models_params_for_tuning = get_folktables_income_models_params_for_tuning(experiment_seed)
# Create a transformed data loader
exp_iter_data_loader = create_experiment_data_loader(data_loader)
exp_iter_data_loader.columns_with_nulls = COLUMNS_TO_TRANSFORM
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)

In [132]:
run_exp_iter_with_models_stress_testing(data_loader=exp_iter_data_loader,
                                        experiment_seed=experiment_seed,
                                        test_set_fraction=TEST_SET_FRACTION,
                                        db_writer_func=db_writer_func,
                                        error_injector=error_injector,
                                        injector_config_lst=INJECTOR_CONFIG_LST,
                                        preprocessor=preprocessor,
                                        models_params_for_tuning=models_params_for_tuning,
                                        metrics_computation_config=metrics_computation_config,
                                        custom_table_fields_dct=custom_table_fields_dct,
                                        with_tuning=True,
                                        tuned_params_df_path=None,
#                                         with_tuning=False,
#                                         tuned_params_df_path=tuned_params_df_path,
                                        save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                        verbose=True)

2023-04-23 23:17:11 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'error_percentages': '0%,10%,20%,40%,60%,80%,100%',
 'error_type': 'Random Nulls',
 'experiment_iteration': 'Exp_iter_2',
 'injector_config_lst': '[0.1, 0.2, 0.4, 0.6, 0.8, 1.0]',
 'model_init_seed': 200,
 'preprocessing_technique': 'cat: mode_trimmed_0.3, num: median',
 'session_uuid': 'a4febf95-81bf-4aee-9b51-a690d596d5d2'}


[CV 2/3; 1/4] START criterion=gini, max_depth=20, max_features=sqrt, min_samples_split=0.1
[CV 2/3; 1/4] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_split=0.1; Accuracy_Score: (test=0.747) F1_Score: (test=0.727) total time=   0.1s
[CV 1/3; 4/4] START criterion=entropy, max_depth=30, max_features=sqrt, min_samples_split=0.1
[CV 1/3; 4/4] END criterion=entropy, max_depth=30, max_features=sqrt, min_samples_split=0.1; Accuracy_Score: (test=0.745) F1_Score: (test=0.729) total time=   0.1s
[CV 3/3; 4/4] START criterion=entropy, max_depth=30, max_features=sqrt, min_samples_split=0.1
[CV 3/3; 4/4] END criterion=entropy, 

2023-04-23 23:17:13 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([20518, 20570, 38530, 39945,  1713, 19318, 15625, 36291, 26020,
            18954,  7039, 40519, 42300,  5620, 39590, 39048, 33565, 18881,
            43190,  5750],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([20518, 20570, 38530, 39945,  1713, 19318, 15625, 36291, 26020,
            18954,  7039, 40519, 42300,  5620, 39590, 39048, 33565, 18881,
            43190,  5750],
           dtype='int64')
transformed_X_test.isnull().sum() --  MAR         42
MIL         55
ESP         48
MIG         51
DREM        49
NATIVITY    53
DIS         65
DEAR        70
DEYE        62
SEX          0
RAC1P        0
RELP        62
CIT         69
ANC         65
SCHL        52
AGEP         0
dtype: int64
transformed_X_test.isnull().sum() --  MAR         142
MIL         156
ESP         141
MIG         149
DREM        152
NATIVITY    150
DIS         150
DEAR        148
DEYE        146
SEX           0
RAC1P

2023-04-23 23:19:23 experiment_interface.py INFO    : Models are tuned and saved to a file


...

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/2 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

### Experiment iteration 3

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filename = 'tuning_results_COMPAS_exp_iter_1_20230425__164903.csv'
tuned_params_df_path = os.path.join(SAVE_RESULTS_DIR_PATH, 'models_tuning', tuned_params_filename)
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

error_injector = RandomNullsInjectorV2(experiment_seed, columns_to_transform=COLUMNS_TO_TRANSFORM, row_idx_nulls_percentage=0.0,
                                       max_num_columns_to_effect=MAX_NUM_COLUMNS_TO_EFFECT)
models_params_for_tuning = get_folktables_income_models_params_for_tuning(experiment_seed)
# Create a transformed data loader
exp_iter_data_loader = create_experiment_data_loader(data_loader)
exp_iter_data_loader.columns_with_nulls = COLUMNS_TO_TRANSFORM
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_models_stress_testing(data_loader=exp_iter_data_loader,
                                        experiment_seed=experiment_seed,
                                        test_set_fraction=TEST_SET_FRACTION,
                                        db_writer_func=db_writer_func,
                                        error_injector=error_injector,
                                        injector_config_lst=INJECTOR_CONFIG_LST,
                                        preprocessor=preprocessor,
                                        models_params_for_tuning=models_params_for_tuning,
                                        metrics_computation_config=metrics_computation_config,
                                        custom_table_fields_dct=custom_table_fields_dct,
                                        with_tuning=True,
                                        tuned_params_df_path=None,
#                                         with_tuning=False,
#                                         tuned_params_df_path=tuned_params_df_path,
                                        save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                        verbose=True)

### Experiment iteration 4

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filename = 'tuning_results_COMPAS_exp_iter_1_20230425__164903.csv'
tuned_params_df_path = os.path.join(SAVE_RESULTS_DIR_PATH, 'models_tuning', tuned_params_filename)
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

error_injector = RandomNullsInjectorV2(experiment_seed, columns_to_transform=COLUMNS_TO_TRANSFORM, row_idx_nulls_percentage=0.0,
                                       max_num_columns_to_effect=MAX_NUM_COLUMNS_TO_EFFECT)
models_params_for_tuning = get_folktables_income_models_params_for_tuning(experiment_seed)
# Create a transformed data loader
exp_iter_data_loader = create_experiment_data_loader(data_loader)
exp_iter_data_loader.columns_with_nulls = COLUMNS_TO_TRANSFORM
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_models_stress_testing(data_loader=exp_iter_data_loader,
                                        experiment_seed=experiment_seed,
                                        test_set_fraction=TEST_SET_FRACTION,
                                        db_writer_func=db_writer_func,
                                        error_injector=error_injector,
                                        injector_config_lst=INJECTOR_CONFIG_LST,
                                        preprocessor=preprocessor,
                                        models_params_for_tuning=models_params_for_tuning,
                                        metrics_computation_config=metrics_computation_config,
                                        custom_table_fields_dct=custom_table_fields_dct,
                                        with_tuning=True,
                                        tuned_params_df_path=None,
#                                         with_tuning=False,
#                                         tuned_params_df_path=tuned_params_df_path,
                                        save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                        verbose=True)

### Experiment iteration 5

In [15]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filename = 'tuning_results_COMPAS_exp_iter_1_20230425__164903.csv'
tuned_params_df_path = os.path.join(SAVE_RESULTS_DIR_PATH, 'models_tuning', tuned_params_filename)
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

error_injector = RandomNullsInjectorV2(experiment_seed, columns_to_transform=COLUMNS_TO_TRANSFORM, row_idx_nulls_percentage=0.0,
                                       max_num_columns_to_effect=MAX_NUM_COLUMNS_TO_EFFECT)
models_params_for_tuning = get_folktables_income_models_params_for_tuning(experiment_seed)
# Create a transformed data loader
exp_iter_data_loader = create_experiment_data_loader(data_loader)
exp_iter_data_loader.columns_with_nulls = COLUMNS_TO_TRANSFORM
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)

In [16]:
run_exp_iter_with_models_stress_testing(data_loader=exp_iter_data_loader,
                                        experiment_seed=experiment_seed,
                                        test_set_fraction=TEST_SET_FRACTION,
                                        db_writer_func=db_writer_func,
                                        error_injector=error_injector,
                                        injector_config_lst=INJECTOR_CONFIG_LST,
                                        preprocessor=preprocessor,
                                        models_params_for_tuning=models_params_for_tuning,
                                        metrics_computation_config=metrics_computation_config,
                                        custom_table_fields_dct=custom_table_fields_dct,
                                        with_tuning=True,
                                        tuned_params_df_path=None,
#                                         with_tuning=False,
#                                         tuned_params_df_path=tuned_params_df_path,
                                        save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                        verbose=True)

2023-04-25 20:24:40 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 500,
 'error_percentages': '0%,5%,10%,20%,30%,40%,50%',
 'error_type': 'Random Nulls',
 'experiment_iteration': 'Exp_iter_5',
 'injector_config_lst': '[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]',
 'model_init_seed': 500,
 'preprocessing_technique': 'cat: mode, num: median',
 'session_uuid': '340a6ce1-3261-4b64-a083-d74dfaad97cf'}




2023-04-25 20:24:40 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([  851, 17635,  1369, 18670, 19789,  1799,  8446, 14863,  4634,
             7378,  9366, 16042,  1195,   800,  1758,   548, 17791,  2416,
             6478, 11097],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([  851, 17635,  1369, 18670, 19789,  1799,  8446, 14863,  4634,
             7378,  9366, 16042,  1195,   800,  1758,   548, 17791,  2416,
             6478, 11097],
           dtype='int64')
error_injector.seed --  501
transformed_X_test:
 SCHL     29
COW      28
MAR      32
OCCP     24
POBP     33
RELP     28
SEX       0
RAC1P     0
AGEP      0
WKHP     25
dtype: int64
error_injector.seed --  502
transformed_X_test:
 SCHL     55
COW      49
MAR      45
OCCP     65
POBP     60
RELP     59
SEX       0
RAC1P     0
AGEP      0
WKHP     58
dtype: int64
error_injector.seed --  503
transformed_X_test:
 SCHL     115
COW      102
MAR      113
OCCP     126
POBP     115
RELP     113
SEX

2023-04-25 20:33:52 experiment_interface.py INFO    : Models are tuned and saved to a file


...

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x14ac815015e0>
Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x14ac815015e0>
Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x14acc3f13b80>
Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x14ac815018b0>
Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x14acc3f06d30>
Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x

In [54]:
# client.close()